# 多输入通道

In [2]:
import torch
from d2l import torch as d2l

In [3]:
def corr2d_multi_in(X,K):
    #先遍历X和K的第0个维度（通道维度），再把他们加到一起
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [34]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

# 多输出通道

In [54]:
def corr2d_multi_in_out(X,K):
    #迭代K的第0个维度，每次都对X执行互相关运算
    #最后将所有的结果都加到一起
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

Q=torch.stack([K,K+1,K+2],0)
corr2d_multi_in_out(X,Q)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 1×1 卷积层

In [59]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w =X.shape
    c_0 =K.shape[0]
    X=X.reshape(shape=(c_i,h*w))
    K=K.reshape(shape=(c_0,c_i))
    #全连接层中的矩阵乘法
    Y=torch.matmul(K,X)
    return Y.reshape((c_0,h,w))

In [60]:
X=torch.normal(0,1,(3,3,3))
K=torch.normal(0,1,(2,3,1,1))

Y1=corr2d_multi_in_out_1x1(X,K)
Y2=corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum())<1e-6